In [36]:
import os
import pickle
import numpy as np
#import keras.preprocessing
from keras.applications import xception
from keras.preprocessing import image
from keras.models import load_model, model_from_json
import json
import cv2

In [37]:
with open('/Users/emilygeller/ds/metis/metisgh/Geller_Metis/Project 5/seedlings_model.txt') as jsonfile:
    json_string = json.load(jsonfile)

In [38]:
model = model_from_json(json_string)
model.load_weights('/Users/emilygeller/ds/metis/metisgh/Geller_Metis/Project 5/xception_seedlings_weights.h5')

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 12)                3084      
Total params: 2,363,660
Trainable params: 2,363,660
Non-trainable params: 0
_________________________________________________________________


In [40]:
## detect and segment plants in the image 

def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    sensitivity = 35
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

def read_segmented_image(filepath, img_size):
    img = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img = cv2.resize(img.copy(), img_size, interpolation = cv2.INTER_AREA)

    image_mask = create_mask_for_plant(img)
    image_segmented = segment_plant(img)
    image_sharpen = sharpen_image(image_segmented)
    return img, image_mask, image_segmented, image_sharpen

In [41]:
def preprocess_image(img):
    img /= 255.
    img -= 0.5
    img *= 2
    return img

In [42]:
CATEGORIES = [['Black-grass','Weed'], ['Charlock','Weed'] , ['Cleavers','Weed'], ['Common Chickweed','Weed'], ['Common wheat','Crop'], ['Fat Hen','Weed'], ['Loose Silky-bent','Weed'],
              ['Maize','Crop'], ['Scentless Mayweed','Weed'], ['Shepherds Purse','Weed'], ['Small-flowered Cranesbill','Weed'], ['Sugar beet','Crop']]

In [43]:
# load xception base model and predict the last layer, resulting in 2048 neurons per image
base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
base_model.compile('sgd','mse')

In [44]:
def identify(filepath):

    _,_,_,img = read_segmented_image(filepath, (299,299))
    img = preprocess_image(np.expand_dims(img.copy().astype(np.float), axis=0))
    img_bf = base_model.predict(img)
    return CATEGORIES[np.argmax(model.predict(img_bf))]

In [47]:
identify('/Users/emilygeller/ds/metis/metisgh/Geller_Metis/Project 5/app_logreg/static/seedling02.png')

['Charlock', 'Weed']